# XGB kaggle comp

In [1]:
# Load basic libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statistics as stats
import time

# XGB libraries
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform, randint
from sklearn.preprocessing import StandardScaler


# Import data
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
sample_sub =  pd.read_csv("data/sample_submission.csv")

# Remove NA column from training data
train_df = train_df.drop(columns='Unnamed: 12')

# Fix column name error
test_df = test_df.rename(columns={'TA1':'TA1.x'})

In [2]:
# Assign features
X = train_df.drop(columns=['id', 'DIC'], axis=1)
y = train_df['DIC']

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=808) 

# Scale the data
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns=X.columns)

In [4]:
# Determine best number of trees using early stopping
xgb = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.1, 
    early_stopping_rounds=50, 
    eval_metric="rmse",  # Use RMSE for regression
    random_state=808)

# Fit model
xgb.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=0)

# Print best number of trees
best_ntrees = xgb.best_iteration
print(f"Best number of trees {best_ntrees}")

Best number of trees 305


In [5]:
# Initialize second XGB to tune learning rate
xgb2 = XGBRegressor(
    n_estimators=best_ntrees,
    early_stopping_rounds=50, 
    eval_metric="rmse", 
    random_state=808)

# Fit model
xgb2.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=0)

# Create a parameter distribution for learning rate
param_dist = {
    "learning_rate": uniform(0.01, 0.3), 
}

# Set up RandomizedSearchCV
rs = RandomizedSearchCV(
    xgb2, param_dist, n_iter=20, scoring='neg_root_mean_squared_error', 
    cv=3, verbose=0, random_state=808, n_jobs=8
)

# Fit random search
rs.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=0)

# Print best number of learners
best_lr = rs.best_params_['learning_rate']
print(f"Best learning rate: {best_lr:.4f}")

Best learning rate: 0.1561


In [6]:
# Tune tree specific parameters
xgb3 = XGBRegressor(
    n_estimators = best_ntrees,
    learning_rate = best_lr, 
    early_stopping_rounds=50, 
    eval_metric="rmse", 
    random_state=808)

# Fit model
xgb3.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=0)

# Second param dist
param_dist2 = {
    "max_depth": randint(3, 10), 
    "min_child_weight": randint(1, 10),
    "gamma": uniform(0.05, 0.05)
}

# Set up RandomizedSearchCV
rs2 = RandomizedSearchCV(
    xgb3, param_dist2, 
    n_iter=20, scoring='neg_root_mean_squared_error', 
    cv=3, verbose=False, random_state=808, n_jobs=10
)

# Run random search
rs2.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=0)

# Print best tree parameters
best_tree_params = rs2.best_params_
print(f"Best tree parameters: {best_tree_params}")

Best tree parameters: {'gamma': 0.06711966552658265, 'max_depth': 5, 'min_child_weight': 3}


In [7]:
# Tune stochastic components
xgb4 = XGBRegressor(
    n_estimators=best_ntrees,
    learning_rate=best_lr,
    **best_tree_params, 
    early_stopping_rounds=50, 
    eval_metric="rmse", 
    random_state=808)

# Fit model
xgb4.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=0)

# Third param dist
param_dist3 = {
    "subsample": uniform(0.5, 0.5),
    "colsample_bytree": uniform(0.5, 0.5) 
}

# Set up RandomizedSearchCV
rs3 = RandomizedSearchCV(
    xgb4, param_dist3, n_iter=20, scoring='neg_root_mean_squared_error', 
    cv=3, verbose=False, random_state=808, n_jobs=10
)

# Run random search
rs3.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=0)

# Print best stochastic parameters
best_stochastic_params = rs3.best_params_
print(f"Best stochastic parameters: {best_stochastic_params}")

Best stochastic parameters: {'colsample_bytree': 0.6673574366507202, 'subsample': 0.8046025228990683}


In [9]:
# Initialize fifth XGB model
xgb5 = XGBRegressor(
    n_estimators=best_ntrees,
    learning_rate=best_lr,
    **best_tree_params, 
    **best_stochastic_params, 
    early_stopping_rounds=50, 
    eval_metric="rmse", 
    random_state=808)

# Fit model
xgb5.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=0)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6673574366507202, device=None,
             early_stopping_rounds=50, enable_categorical=False,
             eval_metric='rmse', feature_types=None, gamma=0.06711966552658265,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.15614429901042864,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=305, n_jobs=None,
             num_parallel_tree=None, random_state=808, ...)

In [ ]:
# Predict on validation and testing data
y_pred_val = xgb5.predict(X_val_scaled)

# Calculate Mean Squared Error (MSE)
val_mse = mean_squared_error(y_val, y_pred_val)

# Calculate Root Mean Squared Error (RMSE)
val_rmse = np.sqrt(val_mse)

# Print results
print(f"Validation RMSE: {val_rmse:.4f}, MSE: {val_mse:.4f}")


Validation RMSE: 6.8878, MSE: 47.4414


In [ ]:
# Get feature importance
feat_imp = pd.DataFrame({'Feature': X_train_scaled.columns, 'Importance': xgb5.feature_importances_})

# Sort by importance
feat_imp = feat_imp.sort_values(by="Importance", ascending=False)
feat_imp

,Feature,Importance
12,PO4uM,0.494485
13,SiO3uM,0.370204
11,R_Oxy_micromol.Kg,0.082770
7,R_Depth,0.015209
8,R_Sal,0.011374
14,TA1.x,0.011063
4,NO3uM,0.010024
15,Salinity1,0.003528
6,R_TEMP,0.000303
3,NO2uM,0.000191


## Trying `hyperopt`

[Documentation here](https://hyperopt.github.io/hyperopt/)

In [3]:
# Define objective function to minimize
def objective(params):
    model = XGBRegressor(
        n_estimators=int(params["n_estimators"]),
        learning_rate=params["learning_rate"],
        max_depth=int(params["max_depth"]),
        min_child_weight=params["min_child_weight"],
        subsample=params["subsample"],
        colsample_bytree=params["colsample_bytree"],
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        early_stopping_rounds=50,
        random_state=808
    )
    
    # Fit model
    model.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=0)

    # Make predictions
    y_pred = model.predict(X_val_scaled)

    # Calculate rmse
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    
    return {'loss': rmse, 'status': STATUS_OK}

# Create hyperparameter space
space = {
    "n_estimators": hp.quniform("n_estimators", 100, 1200, 10),
    "learning_rate": hp.uniform("learning_rate", 0.005, 0.3),
    "max_depth": hp.quniform("max_depth", 3, 20, 1),
    "min_child_weight": hp.uniform("min_child_weight", 1, 10),
    "subsample": hp.uniform("subsample", 0.5, 1.0),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1.0),
    "gamma": hp.uniform("gamma", 0, 10),  
    "reg_alpha": hp.uniform("reg_alpha", 0, 1),  
    "reg_lambda": hp.uniform("reg_lambda", 0, 1),  
}

# Run hyperopt 
trials = Trials()
best_params = fmin(
    fn=objective, 
    space=space,      
    algo=tpe.suggest,   
    max_evals=100, 
    trials=trials,       
    rstate=np.random.default_rng(808)  
)

# Print results
print("Best Hyperparameters:", best_params)

100%|██████████| 100/100 [00:45<00:00,  2.21trial/s, best loss: 6.578703365432439]
Best Hyperparameters: {'colsample_bytree': 0.9302942397848147, 'gamma': 4.344605352493377, 'learning_rate': 0.06914991345308155, 'max_depth': 5.0, 'min_child_weight': 1.86348125935485, 'n_estimators': 880.0, 'reg_alpha': 0.9922135764384623, 'reg_lambda': 0.003935981852993958, 'subsample': 0.619839945286353}


In [4]:
# Convert int hyperparameters to fix type error
best_params["n_estimators"] = int(best_params["n_estimators"])
best_params["max_depth"] = int(best_params["max_depth"])

# Initialize best hyperopt model
xgb_hyper = XGBRegressor(**best_params, eval_metric='rmse', random_state=808)

# Fit model
xgb_hyper.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=0)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9302942397848147, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='rmse', feature_types=None, gamma=4.344605352493377,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.06914991345308155,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1.86348125935485, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=880,
             n_jobs=None, num_parallel_tree=None, random_state=808, ...)

In [10]:
# Predict on validation  data
y_pred_hyper = xgb_hyper.predict(X_val_scaled)

# Calculate rmse
hyper_rmse = np.sqrt(mean_squared_error(y_val, y_pred_hyper))

# Print results
print(f"Hyperopt Validation RMSE: {hyper_rmse:.4f}")


Hyperopt Validation RMSE: 5.0125


In [6]:
# Get feature importance
feat_imp_hyper = pd.DataFrame({'Feature': X_train_scaled.columns, 'Importance': xgb_hyper.feature_importances_})

# Sort by importance
feat_imp_hyper = feat_imp_hyper.sort_values(by="Importance", ascending=False)
feat_imp_hyper

,Feature,Importance
12,SiO3uM,0.603358
11,PO4uM,0.224232
10,R_Oxy_micromol.Kg,0.096311
3,NO3uM,0.034763
14,Salinity1,0.012545
13,TA1.x,0.011401
7,R_Sal,0.007707
6,R_Depth,0.005928
2,NO2uM,0.001661
9,R_Nuts,0.000480


In [7]:
# Re-fit model with the entirety of the training data
X_total = train_df.drop(columns=['id', 'DIC'], axis=1)
y_total = train_df['DIC']
X_test = test_df.drop(columns=['id'], axis=1) 


# Scale x_total and test_df
scaler = StandardScaler()
X_total_scaled = pd.DataFrame(scaler.fit_transform(X_total), columns=X_total.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

# Fit model
xgb_hyper.fit(X_total_scaled, y_total)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9302942397848147, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='rmse', feature_types=None, gamma=4.344605352493377,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.06914991345308155,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1.86348125935485, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=880,
             n_jobs=None, num_parallel_tree=None, random_state=808, ...)

In [8]:
# Generate predictions on scaled test data
y_pred_total = xgb_hyper.predict(X_test_scaled)

# Add DIC to test dataset
test_df['DIC'] = y_pred_total
submission = test_df[['id', 'DIC']]
submission.head()

,id,DIC
0,1455,2168.957031
1,1456,2194.675537
2,1457,2327.310303
3,1458,1989.420410
4,1459,2150.913574


In [ ]:
# Export for submission
# submission.to_csv('submission.csv', index=False)